In [1]:
from google.colab import drive
drive.mount('/content/drive')
#ls drive/MyDrive/chat_api/dataset.xlsx

Mounted at /content/drive


# Read the data

In [4]:
import pandas as pd
import os

folder_path = 'drive/MyDrive/chat_api/metrics_corr'
file_names = [
    "gpt4turbo_rus_results.xlsx",
    "gpt4turbo_en_results.xlsx",
    "gpt4o_rus_results.xlsx",
    "gpt4o_en_results.xlsx",
    "gpt3_5_cmpre_results_rus_new.xlsx",
    "gpt3_5_cmpre_results_en.xlsx",
    "gemini_cmpre_en_results.xlsx",
    "gemini_cmpre_results_rus.xlsx",
    "mistral_en_results.xlsx",
    "mistral_ru_results.xlsx",
    "llama_en.xlsx",
    "llama_ru.xlsx"
]

data_frames = {}
for file_name in file_names:
    file_path = os.path.join(folder_path, file_name)
    try:
        data_frames[file_name] = pd.read_excel(file_path)
        print(f"File {file_name} read successfully.")
    except Exception as e:
        print(f"File reading error {file_name}: {e}")
data_frames["gpt4o_en_results.xlsx"].head()

File gpt4turbo_rus_results.xlsx read successfully.
File gpt4turbo_en_results.xlsx read successfully.
File gpt4o_rus_results.xlsx read successfully.
File gpt4o_en_results.xlsx read successfully.
File gpt3_5_cmpre_results_rus_new.xlsx read successfully.
File gpt3_5_cmpre_results_en.xlsx read successfully.
File gemini_cmpre_en_results.xlsx read successfully.
File gemini_cmpre_results_rus.xlsx read successfully.
File mistral_en_results.xlsx read successfully.
File mistral_ru_results.xlsx read successfully.
File llama_en.xlsx read successfully.
File llama_ru.xlsx read successfully.


,section,question_etalon,solution_etalon,option_etalon,question_llm,solution_llm,BERTScore_F1,ROUGE1_F1,ROUGE2_F1
0,Algebra,What is the number that ends the expression 9^121,"Let's start with the fact that 9^1 = 9, 9^2 = ...",A) 1 B) 7 C) 3 D) 9 E) 5,What is the number that ends the expression 9^121,We are tasked with finding the last digit of 9...,0.859652,0.344262,0.173554
1,Logic,What number corresponds to the question mark: ...,The pattern of this series is as follows: 2 is...,A) 155 B) 75 C) 53 D) 99 E) 57,What number corresponds to the question mark: ...,"We are given the sequence: 3, 5, 15, 17, 51, a...",0.845529,0.424528,0.114286
2,Logic,What number should be in place of the question...,"To complete this task, we need to add a number...",A) 15 B) 25 C) 33 D) 45 E) 43,What number should be in place of the question...,Let's break down the given pattern step by ste...,0.844169,0.290076,0.107692
3,Algebra,The Algebra mean of six numbers is 70 and the ...,"Let's denote the sum of six numbers as S1, and...",A) 85 B) 82 C) 17 D) 14 E) 36,The Algebra mean of six numbers is 70 and the ...,"To solve this problem, we need to find the ove...",0.855891,0.388889,0.186916
4,Logic,Replace the letters with digits so that the re...,"The addends are four-digit numbers, and the su...",A) the product of different digits is a multip...,Replace the letters with digits so that the re...,"We are given the equation УДАР + УДАР = ДРАКА,...",0.816908,0.334559,0.084871


In [5]:
data_frames.keys()

dict_keys(['gpt4turbo_rus_results.xlsx', 'gpt4turbo_en_results.xlsx', 'gpt4o_rus_results.xlsx', 'gpt4o_en_results.xlsx', 'gpt3_5_cmpre_results_rus_new.xlsx', 'gpt3_5_cmpre_results_en.xlsx', 'gemini_cmpre_en_results.xlsx', 'gemini_cmpre_results_rus.xlsx', 'mistral_en_results.xlsx', 'mistral_ru_results.xlsx', 'llama_en.xlsx', 'llama_ru.xlsx'])

In [ ]:
gpt4turbo_ru = data_frames["gpt4turbo_rus_results.xlsx"]
gpt4turbo_en = data_frames["gpt4turbo_en_results.xlsx"]
gpt4o_ru = data_frames["gpt4o_rus_results.xlsx"]
gpt4o_en = data_frames["gpt4o_en_results.xlsx"]
gpt35_ru = data_frames["gpt3_5_cmpre_results_rus_new.xlsx"]
gpt35_en = data_frames["gpt3_5_cmpre_results_en.xlsx"]
gemini_en = data_frames["gemini_cmpre_en_results.xlsx"]
gemini_ru = data_frames["gemini_cmpre_results_rus.xlsx"]
mistral_en = data_frames["mistral_en_results.xlsx"]
mistral_ru = data_frames["mistral_ru_results.xlsx"]
llama_en = data_frames["llama_en.xlsx"]
llama_ru = data_frames["llama_ru.xlsx"]

# For each dataset do the same

In [ ]:
import re
import pandas as pd

def extract_answer(text):
    if pd.isna(text):
        return None

    cyr_to_lat = {
        'А': 'A', 'Б': 'B', 'В': 'B', 'С': 'C', 'Д': 'D', 'Е': 'E', 'Г': 'D',
        'а': 'A', 'б': 'B', 'в': 'B', 'с': 'C', 'д': 'D', 'е': 'E', 'г': 'D'
    }

    # 1. Search for explicit answers (Cyrillic + Latin)
    answer_matches = list(re.finditer(
        r'(?i)\b(?:ответ|вариант|итог|правильный|answer\s+is|answer\s*:)\b\s*[:—\-]*\s*\(?\b([АБВСДЕГA-E])\b\)?',
        text
    ))

    if answer_matches:
        # We take the last result found
        last_match = answer_matches[-1]
        letter = last_match.group(1).upper()
        # We convert Cyrillic, leave Latin as is
        return cyr_to_lat.get(letter, letter)

    # 2. Search for the last letter (Cyrillic/Latin)
    last_sentence = re.split(r'[.!?]', text)[-1]
    last_char = re.search(r'\b([АБВСДЕГA-E])\b', last_sentence.upper())
    if last_char:
        letter = last_char.group(1).upper()
        return cyr_to_lat.get(letter, letter)

    # 3. Search after the very last parenthesis if nothing is found
    last_bracket_match = re.search(r'\(([^()]*)\)[^()]*$', text)
    if last_bracket_match:
        letter = last_bracket_match.group(1).strip().upper()
        return cyr_to_lat.get(letter, letter)

    # 4. If nothing is found, return None
    return None

# Apply and save
mistral_ru['extracted_letter'] = mistral_ru['solution_llm'].apply(extract_answer)
mistral_ru.to_excel('/content/drive/My Drive/chat_api/get_option/mistral_ru.xlsx', index=False, engine='openpyxl')